In [ ]:
You are required to:

Read in the data from Sheet 1 using Pandas and perform the necessary data wrangling to prepare it. Then you should complete the user stories below:

User Stories: 

#1: When I choose a social media network, I am able to get a view the number of users in each country.


#2: When I choose a country, I am able to see all the relevant social networks its people use ranked in descending order. 

#3: I can view a comparison of the percentage of users based on the population for each social media network by the population of the country. (You will have to scrape the population data and provide the source(s).)


In [348]:
# import and clean data
import pandas as pd

df = pd.read_excel('SocMed_Geographic.xlsx',sheet_name='Sheet1',header=1)      
df = df.fillna(0)
df = df.set_index('Unnamed: 0')
df = df.reset_index()
clean = df.rename(columns={'Unnamed: 0':'Platform'},inplace=False)
clean = clean.set_index('Platform')
# removing whitespaces
clean.columns = clean.columns.str.strip()
clean.index = clean.index.str.strip()
clean = clean.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
clean = clean.apply(lambda x: x.str.replace(',','') if x.dtype == "object" else x)
clean = clean.fillna(0)

In [349]:
clean

,United States,Indonesia,Singapore,China,India,Vietnam,Philippines,Bangladesh
Platform,,,,,,,,
Bilibili,0,0,0,27400000 (2019 30),0,0,0,0
Facebook,190000000 (2020 31),120000000 (2020 31),4450000 (2020 32),3000000 (2020 36),280000000 (2020 31),63000000 (2020 31),72000000 (2020 31),37000000 (2020 31)
Instagram,107000000 (2020 33),59840000 (2020 34),2110000 (2020 35),3600000 (2020 37),88000000 (2020 38),8000000 (2019 39),1100000 (2020 38),1 676 100 (2019 40)
Linkedln,160000000 (2020 41),15240000 (2020 42),2 682 000 (2019 43),5397000 (2019 44),62000000 (2020 45),3 087 500 (2019 46),2773485 (2019 47),2 990 000 (2019 48)
Tencent QQ,0,0,0,659100000 (2020 49),0,0,0,0
Tik Tok,45400000 (2020 50),81000000 (2020 51),800000‬ (2020 52),400000000 (2020 53),120000000 (2019 54),12000000 (2019 55),0,0
Twitter,55700000 (2019 56),11800000 (2020 57),1370000 (2020 58),3200000 (2020 59),13150000 (2020 57),127000000 (2020 60),10400000 (2019 61),3000000 (2020 62)
WeChat,0,0,1000000 (2020 63),900000000‬ (2019 64),0,0,0,0
Weibo,0,0,800000 (2019 65),480000000 (2019 66),0,600000,0,0


In [257]:
# loading website with population data
from bs4 import BeautifulSoup
import requests

url = 'https://www.worldometers.info/world-population/population-by-country/'

def read_url(url):
    response = requests.get(url)
    page_content = response.text
    return page_content
    
page_content = read_url(url)
soup = BeautifulSoup(page_content, 'html.parser')
data_items = soup.find_all('a')

In [355]:
# coding choosing interface
import plotly.express as px

user_story = input('What would you like to do? (Type command from list)\nCheck Social Media\nCheck Country\nCheck Percentage\n')

# user story 1
if user_story == 'Check Social Media':
    for row in clean.index:
        print('"'+row+'"')
    platform = input('\nWhich platform are you interested in? (Type platform name)\n')
    data = clean.loc[platform] # get data
    country = {} # get dictionary
    for i in range(len(data)):
        name = clean.loc[[platform]].columns[i]
        num = str(data[i])
        if '(' in num:
            num = num.split('(')[0]
        country[name] = num
    country = dict(sorted(country.items(), key=lambda item: item[1]))
    keys = list(country.keys())
    values = list(country.values())
    fig = px.bar(y=keys, x=values,
                 labels={'x': 'Number of Users', 'y': 'Country'},
                 orientation='h', 
                 title= 'Total users by Country')
    fig.show()
    
# user story 2
elif user_story == 'Check Country':
    for col in clean.columns:
        print('"'+col+'"')
    country = input('\nWhich country are you interestesd in? (Type country name)\n')
    data = clean[country] # get data
    platform = {} # get dictionary
    for i in range(len(data)):
        name = clean[[country]].index[i]
        num = str(data[i])
        if '(' in num:
            num = num.split('(')[0]
        platform[name] = num
    platform = dict(sorted(platform.items(), key=lambda item: item[1]))
    keys = list(platform.keys())
    values = list(platform.values())
    fig = px.bar(y=keys, x=values,
                 labels={'x': 'Number of Users', 'y': 'Platform'},
                 orientation='h', 
                 title= 'Total users by Platform')
    fig.show()
    
# user story 3
elif user_story == 'Check Percentage':
    for col in clean.columns:
        print('"'+col+'"')
    country = input('\nWhich country are you interestesd in? (Type country name)\n')
    data = clean[country] # get data
    platform = {} # get dictionary
    for i in range(len(data)):
        name = clean[[country]].index[i]
        num = str(data[i])
        platform[name] = num
    platform = dict(sorted(platform.items(), key=lambda item: item[1]))
    website = 'https://www.worldometers.info'
    # get total population
    for link in data_items:
        if country in link:
            website += link.get('href')
            about_country = read_url(website)
            soup2 = BeautifulSoup(about_country , 'html.parser') 
    # calculate percentages
    for k,v in platform.items():
        if v != '0':
            year = v.split('(')[1].split(',')[0]
        for i in soup2.find_all('tr'):
            if year in str(i):
                try:
                    total_population = i.find('strong').text
                except:
                    pass
        user_population = v.split('(')[0]
        # stupid unicode
        user_population = user_population.replace(' ','')
        user_population = user_population.encode("ascii", "ignore")
        user_population = user_population.decode()
        percentage = round(int(user_population)/int(total_population.replace(',','')),2)
        platform[k] = percentage
    platform = dict(sorted(platform.items(), key=lambda item: item[1]))
    keys = list(platform.keys())
    values = list(platform.values())
    fig = px.bar(y=keys, x=values,
                 labels={'x': 'Number of Users (%)', 'y': 'Platform'},
                 orientation='h', 
                 title= 'Total users by Platform by Percentage')
    fig.show()

What would you like to do? (Type command from list)
Check Social Media
Check Country
Check Percentage
Check Country
"United States"
"Indonesia"
"Singapore"
"China"
"India"
"Vietnam"
"Philippines"
"Bangladesh"

Which country are you interestesd in? (Type country name)
Singapore
